In [65]:
from bs4 import BeautifulSoup
import requests
import numpy as np

In [87]:
def scrap_entreprises_from_search(url, verbose=False):
    
    # todo:
    #      check if "?page=" is already in the url
    #      add argument nb_entreprise_max
    #      add argument scrap_webpage_of_entreprise
    
    entreprises = {"links": [], "titles": [], "stars": [], "review_counts": [], "locations": []}
    
    page = 1
    url += "?page="
    done = False
    while not done:
        
        req = requests.get(url + str(page))
        if verbose:
            print(url + str(page))
        soup = BeautifulSoup(req.text, "lxml")
        
        entreprise_count = 0
        
        # possibly 2 blocks of entreprises (most rated and all)
        div_containers = soup.find_all("div", class_="styles_businessUnitCardsContainer__1ggaO")

        for div_container in div_containers:
            # for each block
            div = div_container.find("div", class_="styles_categoryBusinessListWrapper__2H2X5")
            
            if div == None:
                # if we find div with 2H2X5 class it's most rated so we need to enter in this div
                # else it's all we don't have to enter anywhere
                div = div_container
                
            a_balises = div.find_all("a")
            for a_balise in a_balises:
                # for each entreprise, find link, title, stars
                link = a_balise["href"]
                title = a_balise.find("div", class_="styles_businessTitle__1IANo").text
                
                # try to get more info if they have
                try:
                    infos_div = a_balise.find("div", class_="styles_textRating__19_fv").text
                    infos = infos_div.split("\xa0·\xa0")
                    infos[0] = int(infos[0].split()[0])
                    infos[1] = float(infos[1].split()[-1])
                except:
                    if verbose:
                        print("No stars and nb_reviews for", title, link)
                    infos = [np.nan, np.nan]
                
                try:
                    div_locations = a_balise.find("div", class_="styles_location__3JATO")
                    spans_locations = div_locations.find_all("span")
                    location = ""
                    for span_location in spans_locations:
                        location += span_location.text
                except:
                    if verbose:
                        print("No location for", title, link)
                    location = np.nan

                entreprises["links"].append(link)
                entreprises["titles"].append(title)
                entreprises["stars"].append(infos[1])
                entreprises["review_counts"].append(infos[0])
                entreprises["locations"].append(location)

                entreprise_count += 1

        if verbose:   
            print("nb entreprise on this page :", entreprise_count)
        if entreprise_count >= 20:
            page += 1
        else:
            done = True
    return entreprises

In [91]:
entreprises = scrap_entreprises_from_search("https://fr.trustpilot.com/categories/food_beverages_tobacco")
#entreprises = scrap_entreprises_from_search("https://fr.trustpilot.com/categories/agriculture_produce")
#entreprises = scrap_entreprises_from_search("https://fr.trustpilot.com/categories/animals_pets", verbose=False)
    
# 18 + 9 * 20 pour animals_pets

In [92]:
len(entreprises["titles"])

594

In [93]:
df = pd.DataFrame(data=entreprises)

NameError: name 'pd' is not defined

In [ ]:
df